In [2]:
from scipy.stats import ttest_1samp
import numpy as np

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

from scipy.stats import norm, t, kstest, shapiro
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [4]:
# Need conda install -c conda-forge openpyxl
data = pd.read_excel ('AB_Test_Results.xlsx')
data.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


In [5]:
data.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [9]:
data.shape

(10000, 3)

In [10]:
data.VARIANT_NAME.value_counts()

variant    5016
control    4984
Name: VARIANT_NAME, dtype: int64

In [16]:
data[data['REVENUE']>0].describe()

,USER_ID,REVENUE
count,152.000000,152.000000
mean,4862.217105,6.542566
std,2987.566745,17.706752
min,56.000000,0.020000
25%,2177.250000,1.250000
50%,4980.500000,2.420000
75%,7393.250000,4.607500
max,9996.000000,196.010000


In [17]:
data.isna().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

In [20]:
control_group = data['REVENUE'][data['VARIANT_NAME']== 'control']
variant_group = data['REVENUE'][data['VARIANT_NAME']== 'variant']

### Проверка на нормальность

In [21]:
W, p_val = shapiro(control_group)
print("Shapiro-Wilk normality test for X")
print("W =", W)
print("P-value =", p_val)

Shapiro-Wilk normality test for X
W = 0.018331527709960938
P-value = 0.0


In [22]:
W, p_val = shapiro(variant_group)
print("Shapiro-Wilk normality test for X")
print("W =", W)
print("P-value =", p_val)

Shapiro-Wilk normality test for X
W = 0.027033090591430664
P-value = 0.0


<b>Группы обладают нормальным распрделением

In [30]:
import plotly.express as px
fig = px.histogram(data, x='REVENUE', color = 'VARIANT_NAME',title='avg_revenue_distribution', marginal = 'box' , nbins = 5)

fig.show(renderer="colab")
# fig.show()

In [32]:
import bootstrapped.bootstrap as bs
import bootstrapped.compare_functions as bs_compare
import bootstrapped.stats_functions as bs_stats

plt.style.use('ggplot')

In [41]:
s1 = control_group.values

s2 = variant_group.values

b = bs.bootstrap_ab(s1,s2, stat_func=bs_stats.mean,
compare_func = bs_compare.difference,alpha=0.05,num_iterations=10000)
        
print(f'Lower_limit: {b.lower_bound}, \nUpper_limit: {b.upper_bound}')

Lower_limit: -0.04369002466585259, 
Upper_limit: 0.13533430694330312


<b>Различия между группами не достоверны, так как ноль входит в диапазон возможных значений.
<p><b>Вывод: изменения после теста не принимаем